In [53]:
import statsforecast
import polars as pl
batting = (
    pl.scan_parquet("../../parquets/batting.parquet")
    .with_columns(
        pl.col("date").cast(pl.String)
        .str.strptime(pl.Date, "%Y%m%d")
        .dt.year()
        .alias("year")
    )
    .group_by(
        "id", "year"
    )
    .agg(pl.col("b_h").sum(), pl.col("b_ab").sum())
    .with_columns((pl.col("b_h") / pl.col("b_ab")).alias("avg"))
    .filter(pl.col("b_ab") > 100)
    .with_columns(
        pl.count("year").over("id").alias("count")
    )
    .filter(pl.col("count") > 10)
    .select(
        pl.col("id").alias("unique_id"),
        pl.col("year").alias("ds"),
        pl.col("avg").alias("y"),
    )
).collect()
batting

unique_id,ds,y
str,i32,f64
"""sciom001""",1980,0.253731
"""ellib103""",1947,0.316637
"""hartg103""",1941,0.294118
"""gehrc101""",1941,0.222727
"""smitl002""",1984,0.25
…,…,…
"""singk001""",1976,0.277574
"""gutir001""",1998,0.261477
"""altuj001""",2014,0.340909


In [54]:
models = [
    statsforecast.models.AutoARIMA(),
    # statsforecast.models.AutoETS(),
    # statsforecast.models.AutoRegressive(10),
    # statsforecast.models.HoltWinters(),
    # statsforecast.models.HistoricAverage(),
]

In [55]:
# Instantiate StatsForecast class as sf
sf = statsforecast.StatsForecast( 
    models=models,
    freq=1, 
    n_jobs=-1,
    verbose=True,
)

In [56]:
forecasts_df = sf.forecast(df=batting, h=3, level=[90])
forecasts_df.head()

Forecast: 100%|██████████| 800/800 [Elapsed: 00:13]


unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
str,i64,f64,f64,f64
"""aaroh101""",1977,0.231344,0.188532,0.274156
"""aaroh101""",1978,0.230063,0.182198,0.277929
"""aaroh101""",1979,0.230703,0.173066,0.288341
"""abreb001""",2015,0.24812,0.211297,0.284944
"""abreb001""",2016,0.24812,0.196044,0.300197


In [57]:
forecasts_df.filter(pl.col("unique_id") == "freef001")

unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
str,i64,f64,f64,f64
"""freef001""",2025,0.300002,0.265624,0.33438
"""freef001""",2026,0.300002,0.265624,0.33438
"""freef001""",2027,0.300002,0.265624,0.33438
